In [1]:
import os 

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file  

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo",openai_api_key=OPENAI_API_KEY)

In [3]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.load_local("../faiss_index", OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model="text-embedding-3-small"),allow_dangerous_deserialization=True)

In [4]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":6})

In [5]:
from langchain import hub

prompt=hub.pull("rlm/rag-prompt")

/usr/local/python/3.12.1/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [6]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

system_prompt = """ Given the chat history and a recent user question \
    generate a standalone question \
    that can be answered without the context of the chat history. DO NOT answer the question, \
    just reformulate it if needed or otherwise return it as is. """
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

retriever_with_history = create_history_aware_retriever(
llm, retriever, prompt
)


In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question answering tasks \
Use the following pieces of retrieved context to answer the question \
If you don't know the answer, just say that you don't know \
Use three senetences maximum and keep the answer concise. \

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain=create_retrieval_chain(retriever_with_history, question_answer_chain)

In [8]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "How do we protect manatees from boats?"

ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})

chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

print(f"AI: {ai_msg_1['answer']}")

AI: Setting and enforcing vessel speed limits in specific zones where manatees frequent can help protect them from boat strikes. Studies have shown that speed restrictions can reduce manatee deaths caused by vessel collisions. Regular patrolling and enforcing speed limits throughout all areas where manatees are present can further reduce the risk of collisions.
